# Linear Programming with Python and Cplex

This is a simple example of using Python and Cplex to solve a linear programming.

According to [Wikipedia](https://en.wikipedia.org/wiki/Linear_programming), linear programming (LP, also called linear optimization) is a method to achieve the best outcome (such as maximum profit or lowest cost) in a mathematical model whose requirements are represented by linear relationships. Linear programming is a special case of mathematical programming (also known as mathematical optimization).

$$\text{min}_{\boldsymbol{x}}\enspace \boldsymbol{c}^T \boldsymbol{x} $$ <br/>
$$s.t.\enspace \boldsymbol{A}\boldsymbol{x} <= \boldsymbol{b} $$<br/>
$$\boldsymbol{u} >= \boldsymbol{x} >= \boldsymbol{l} $$ <br/>
       
In the above, c, x, b, u and l are vectors whereas A is matrix.

In the original examples (lpex1.py), you can actually input the elements of the matrix A by specifying its location  in the matrix. The function "lprog" below is a wrapper to facilitate our computation with a simplified input format.
       

## 1. Import Packages

In [1]:
from __future__ import print_function
import sys
import cplex
from cplex.exceptions import CplexError

## 2. Data

In [2]:
# data common to all populateby functions
import numpy as np

A = np.array([[-1.0, 1.0, 1.0],
     [1.0, -3.0, 1.0]])
b = [20.0, 30.0]
c = [1.0, 2.0, 3.0]
u = [40.0, cplex.infinity, cplex.infinity]
l = [0, 0, 0]
saveOpt = True
quietOpt = True

## 3. Code

In [3]:
def lprog(A, b, c, u, l, saveOpt, quietOpt):

    noRow, noCol = len(A), len(A[0])
    
    my_prob = cplex.Cplex()
    my_prob.objective.set_sense(my_prob.objective.sense.maximize)

    my_colnames = ["x"+str(i) for i in range(noCol)]
    my_prob.variables.add(obj=c, ub=u, lb=l, names=my_colnames)
    
    A = [[[i for i in range(noCol)], A[j]] for j in range(noRow)]
    my_prob.linear_constraints.add(lin_expr=A, senses=["L" * noRow], rhs=b)
    
    if quietOpt:
        my_prob.set_log_stream(None)
        my_prob.set_error_stream(None)
        my_prob.set_warning_stream(None)
        my_prob.set_results_stream(None)
    
    try:
        my_prob.solve()
        if saveOpt:
            my_prob.write("lprog.lp")
            print('LP saved as lprog.lp')

        x = my_prob.solution.get_values()
        obj = my_prob.solution.get_objective_value()
        return x, obj
    except CplexError as exc:
        print(exc)
        return [], -1

In [4]:
x, obj = lprog(A, b, c, u, l, saveOpt, quietOpt)
print(x)
print(obj)

LP saved as lprog.lp
[40.0, 17.5, 42.5]
202.5


Let's make the problem infeasible and see if the function can handle that.

In [5]:
b = [-99999.0, -999999.0]
x, obj = lprog(A, b, c, u, l, saveOpt, quietOpt)
print(x)
print(obj)

LP saved as lprog.lp
CPLEX Error  1217: No solution exists.
[]
-1
